## HSE natural language processing
### HW 02

In [52]:
import os
import string
import numpy as np

In [51]:
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict

In [30]:
RESOURCES_PATH = './resources'
TRAIN_TEXTS_PATH = os.path.join(RESOURCES_PATH, 'texts_train.txt')
TRAIN_SCORES_PATH = os.path.join(RESOURCES_PATH, 'scores_train.txt')
TEST_INPUT_FILENAME = os.path.join(RESOURCES_PATH, 'test.in')
TEST_OUTPUT_FILENAME = os.path.join(RESOURCES_PATH, 'test.out')

In [27]:
def load_scores():
    with open(TRAIN_SCORES_PATH, 'r') as scores_file:
        return np.array(list(map(int, scores_file.readlines())))

In [28]:
def load_texts():
    with open(TRAIN_TEXTS_PATH, 'r') as texts_file:
        return np.array(texts_file.readlines())

In [35]:
def run_task(evaluator):
    with open(TEST_INPUT_FILENAME, 'r') as input_file:
        with open(TEST_OUTPUT_FILENAME, 'w') as output_file:
            for line in input_file.readlines():
                output_file.write(evaluator(line))
                output_file.write(os.linesep)

#### Base approach: average mark

In [49]:
avg_mark = int(round(load_scores().mean()))
print('Average mark: {}'.format(avg_mark))

Average mark: 8


In [50]:
run_task(lambda sentence: str(avg_mark))

#### Simple approach: avreage unigram mark

In [58]:
marks = defaultdict(lambda: 0)

for text, score in tqdm(zip(load_texts(), load_scores())):
    text = text.lower().replace(os.linesep, '')
    tokens = text.split(string.punctuation)
    print(text)
    break

сериал очень люблю, но академия и земля вызывает у меня отторжение идеей... не люблю, когда принижают ценность человека как личности, даже не смотря на ошибки личности...
